<a href="https://colab.research.google.com/github/pratzz/Diabetes-Readmission-Prediction/blob/main/Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from pandas_profiling import ProfileReport
import os
import time

In [2]:
timestr = time.strftime("%Y%m%d-%H%M%S")
output_path = os.path.join("../output")
data_path = "../data"
data_exploration_path = os.path.join(output_path, "data_exploration")
processed_data_path = os.path.join(data_path, "processed_data")
if not os.path.exists(data_exploration_path):
    os.makedirs(data_exploration_path) 
if not os.path.exists(processed_data_path):
    os.makedirs(processed_data_path)

In [3]:
train_data = pd.read_csv(os.path.join(data_path, "raw_data/assignment_train.csv"))
test_data = pd.read_csv(os.path.join(data_path, "raw_data/assignment_test.csv"))

# train_data.head(10)

In [4]:
train_data.head(10)

,agent_id,3M_all_min,3M_all_max,3M_all_avg,3M_weekly_all_avg,3M_daily_all_avg,7days_all_gap_days,all_gap_7days_last_vs_previous,30days_all_gap_days,all_gap_30days_last_vs_previous,...,all_gtv_last10days_d1,all_gtv_last10days_d2,all_gtv_last10days_d3,all_gtv_last10days_d4,all_gtv_last10days_d5,all_gtv_last10days_d6,all_gtv_last10days_d7,all_gtv_last10days_d8,all_gtv_last10days_d9,business_risk
0,1,1920.0,18000.0,6640.000000,0.000000,0.000000,0,0.0,0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5.866667
1,2,537256.0,736296.0,641281.000000,207171.666700,27622.888890,1,0.0,1,0.000000,...,31580.0,39835.0,14000.0,7200.0,68900.0,0.0,23000.0,59700.0,73608.0,-0.833333
2,3,79820.0,140612.0,102150.000000,66743.916670,8899.188889,3,1.0,9,3.333333,...,2030.0,14550.0,21969.0,0.0,0.0,2044.0,0.0,0.0,26960.0,-1.266667
3,4,12266.0,12266.0,4088.666667,0.833333,0.111111,6,0.0,29,0.966667,...,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,-6.233333
4,5,0.0,0.0,0.000000,0.000000,0.000000,0,0.0,0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-6.233333
5,6,1243.0,1243.0,414.333333,38.666667,5.155556,0,0.0,0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5.866667
6,7,422255.0,578723.0,485400.000000,101542.750000,13539.033330,0,1.0,5,0.000000,...,4484.0,2000.0,27055.0,9170.0,14206.0,37163.0,20662.0,0.0,18656.0,3.366667
7,8,174792.0,678892.0,489119.333300,229710.666700,30628.088890,1,0.0,1,2.000000,...,11934.0,4881.0,4724.0,0.0,36534.0,42238.0,58890.0,149.0,33875.0,1.966667
8,9,208358.0,424651.0,314300.333300,30577.583330,4077.011111,4,4.0,14,1.153846,...,35.0,0.0,0.0,0.0,2000.0,7510.0,0.0,399.0,184.0,1.633333
9,10,202385.0,343188.0,275367.666700,63917.000000,8522.266667,2,0.0,6,7.000000,...,4800.0,7800.0,1510.0,0.0,0.0,8899.0,13660.0,4560.0,9800.0,2.933333


### Data Profiling & Analysis 


In [5]:
profile_report = ProfileReport(train_data, title="Pandas Profiling Report")
profile_report.to_widgets()

In [6]:
profile_report.to_file(os.path.join(data_exploration_path, "profile_report_1__%s.html"%timestr)) 

## Vif_drop_columns

from statsmodels.stats.outliers_influence import variance_inflation_factor    

def calculate_vif_(X, thresh=5.0):
    variables = list(range(X.shape[1]))
    dropped = True
    while dropped:
        dropped = False
        vif = [variance_inflation_factor(X.iloc[:, variables].values, ix)
               for ix in range(X.iloc[:, variables].shape[1])]

        maxloc = vif.index(max(vif))
        if max(vif) > thresh:
            print('dropping \'' + X.iloc[:, variables].columns[maxloc] +
                  '\' at index: ' + str(maxloc))
            del variables[maxloc]
            dropped = True

    print('Remaining variables:')
    print(X.columns[variables])
    return X.iloc[:, variables]

# calculate_vif_(x_train, 5)

vif_keep_col = ['7days_all_gap_days', 'all_gap_7days_last_vs_previous',
       '30days_all_gap_days', 'all_gap_30days_last_vs_previous',
       'mtd_all_gap_days', 'all_gap_mtd_previous_days', 'all_last_day',
       'all_last30_stable', 'all_last30_inc_count', 'all_consistency_index',
       'avg_all_gap_days_d1_10', 'avg_all_gap_days_d11_20',
       'all_gap_days_d1_10_thisvsprev', 'all_gap_days_d11_20_thisvsprev',
       'all_gap_days_d20_31_thisvsprev', 'all_7days_min_thisvs4w',
       'all_7days_trend_vs4weeks', 'all_7days_trend_vs10weeks',
       'all_7days_vslast_month7days', 'all_7days_max_thisvs10w',
       'all_ystrday_vsmin10d', 'all_ystrday_trend_vs10d',
       'all_ystrday_vsdaybfr', 'all_mrr_trend_vs6M', 'all_lst30days_vsprvmnth',
       'all_mtd_vs_min_lst3M', 'all_trend_mtdvs3M_sameday',
       'all_norm_growth_m1', 'all_norm_growth_m2', 'all_norm_growth_m3',
       'all_norm_growth_m4', 'all_norm_growth_m5', 'all_norm_growth_m6',
       'all_norm_growth_index_last', 'all_gtv_last12Months_m12',
       'all_gtv_last10days_d3', 'all_gtv_last10days_d4',
       'all_gtv_last10days_d6', 'all_gtv_last10days_d7',
       'all_gtv_last10days_d8', 'all_gtv_last10days_d9']


x_train = x_train[vif_keep_col]
x_val = x_val[vif_keep_col]

x_val.shape

## Log Transformation log(x - (min(x)-1))¶

for c in vif_keep_col:
    x_train[c] = np.log(x_train[c] - (min(x_train[c]) - 1))
x_train.shape

for c in vif_keep_col:
    x_val[c] = np.log(x_val[c] - (min(x_val[c]) - 1))
x_val.shape

##### Splitting data into train and validation datasets 


In [7]:
target='business_risk'

X = train_data.drop([target],1)
y = train_data[target]
# Divide the training data given to train and validation data frames
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

##### Saving data on disk

In [8]:
x_train.to_csv(os.path.join(processed_data_path, "x_train.csv"),index=False)
x_val.to_csv(os.path.join(processed_data_path, "x_val.csv"),index=False)
y_train.to_csv(os.path.join(processed_data_path, "y_train.csv"),index=False)
y_val.to_csv(os.path.join(processed_data_path, "y_val.csv"),index=False)
test_data.to_csv(os.path.join(processed_data_path, "x_test.csv"),index=False)